<a href="https://colab.research.google.com/github/catafar/myfirstrepository/blob/main/ProyectoParteIII_Farias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto Parte III - Análisis de Enfermedad Cardíaca

Este notebook corresponde a la entrega final del curso Data Science 1. El objetivo es aplicar técnicas de selección de características, entrenamiento de modelos de clasificación y análisis de métricas para extraer conclusiones.

### Dataset utilizado:
Se emplea el dataset de predicción de enfermedad cardíaca, disponible públicamente. El conjunto contiene información médica de pacientes y una variable dependiente binaria (`HeartDisease`) que indica si el paciente tiene o no una condición cardíaca.

---

## 1. Carga y exploración de datos

En primer lugar se carga librería pandas y se importa el dataset, se realiza una revisión básica de la estructura del mismo y se identifican posibles valores faltantes o problemas en las variables.


In [4]:
import pandas as pd
df = pd.read_excel("heart.xls.xlsx")
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


## 2. Limpieza de datos

Revisamos si existen valores nulos y verificamos los tipos de datos de las columnas.

In [5]:
# Verificar datos nulos
print("Valores nulos por columna:")
print(df.isnull().sum())

# Tipos de datos
print("\nTipos de datos:")
print(df.dtypes)


Valores nulos por columna:
Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
HeartDisease      0
dtype: int64

Tipos de datos:
Age                 int64
Sex                object
ChestPainType      object
RestingBP           int64
Cholesterol         int64
FastingBS           int64
RestingECG         object
MaxHR               int64
ExerciseAngina     object
Oldpeak           float64
ST_Slope           object
HeartDisease        int64
dtype: object


## 3. Separación de variables predictoras y target

Dividimos el dataset en `X` (features o características) y `y` (variable objetivo: `HeartDisease`).


In [8]:
X = df.drop("HeartDisease", axis=1)
y = df["HeartDisease"] #Variable dependiente que queremos predecir con el modelo

## 4. Selección de características (Feature Selection)

Utilizamos el método `SelectKBest` con la prueba chi-cuadrado (`chi2`) para identificar las 8 variables más relevantes. Esta técnica selecciona las variables que tienen una mayor relación estadística con la variable objetivo.

Usamos el método estadístico **chi-cuadrado (chi²)** para medir qué tan relacionada está cada variable con la variable objetivo (`HeartDisease`). Cuanto más fuerte sea la relación, mayor será la puntuación, y por lo tanto, esa variable será más útil para predecir el resultado.


In [10]:
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import MinMaxScaler

# Convertir variables categóricas a numéricas (One-Hot Encoding)
X_encoded = pd.get_dummies(X)

# Escalar X para usar chi2 (que requiere valores positivos)
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_encoded)

# Aplicar SelectKBest para elegir las 8 mejores características
selector = SelectKBest(score_func=chi2, k=8)
X_selected = selector.fit_transform(X_scaled, y)

# Mostrar columnas seleccionadas
selected_features = X_encoded.columns[selector.get_support()]
print("Características seleccionadas:")
print(selected_features)

Características seleccionadas:
Index(['FastingBS', 'Sex_F', 'ChestPainType_ASY', 'ChestPainType_ATA',
       'ExerciseAngina_N', 'ExerciseAngina_Y', 'ST_Slope_Flat', 'ST_Slope_Up'],
      dtype='object')


## 5. División de datos en entrenamiento y test

Dividimos el conjunto de datos en dos partes:
- **Entrenamiento (70%)**: para entrenar el modelo.
- **Prueba (30%)**: para evaluar el rendimiento del modelo con datos que no ha visto.

Esto nos permite validar que el modelo generalice bien.


In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_selected, y, test_size=0.3, random_state=42
)

## 6. Entrenamiento del modelo de clasificación

Entrenamos un modelo de clasificación usando `RandomForestClassifier`. Este modelo es un conjunto de árboles de decisión que mejora la precisión y reduce el riesgo de sobreajuste.

In [12]:
from sklearn.ensemble import RandomForestClassifier

# Entrenar el modelo
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

## 7. Evaluación del modelo

Evaluamos el rendimiento del modelo utilizando métricas de clasificación:
- Precisión (Accuracy)
- Recall (Sensibilidad)
- F1-score
- Matriz de confusión

Estas métricas nos permiten entender qué tan bien predice el modelo tanto los casos positivos como los negativos.


In [13]:
from sklearn.metrics import classification_report, confusion_matrix

# Predecir sobre el conjunto de prueba
y_pred = model.predict(X_test)

# Reporte de métricas
print("Reporte de clasificación:")
print(classification_report(y_test, y_pred))

# Matriz de confusión
print("Matriz de confusión:")
print(confusion_matrix(y_test, y_pred))

Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.79      0.91      0.85       112
           1       0.93      0.84      0.88       164

    accuracy                           0.87       276
   macro avg       0.86      0.87      0.86       276
weighted avg       0.87      0.87      0.87       276

Matriz de confusión:
[[102  10]
 [ 27 137]]


## 8. Análisis de resultados

El modelo `RandomForestClassifier` alcanzó una precisión general del 87% sobre el conjunto de prueba. A continuación se presenta la matriz de confusión para analizar más claramente los aciertos y errores del modelo:

### Matriz de confusión

|                        | Predicho No | Predicho Sí |
|------------------------|-------------|-------------|
| Realmente No (0)       | 102         | 10          |
| Realmente Sí (1)       | 27          | 137         |

Se detectaron correctamente 137 casos positivos y 102 negativos. Hubo 10 falsos positivos (personas sanas clasificadas como enfermas) y 27 falsos negativos (casos de enfermedad no detectados). Aunque el modelo se desempeña bien en general, reducir los falsos negativos sería importante en un contexto clínico.

---

## 9. Conclusiones

Se seleccionaron ocho variables relevantes mediante el método chi-cuadrado, lo que permitió reducir la dimensionalidad del conjunto de datos y mejorar la eficiencia del modelo. El clasificador Random Forest demostró un buen rendimiento general, especialmente en la identificación de casos positivos.


